In [ ]:
import numpy as np
import json
from tqdm import tqdm
import random
from os.path import exists

import torch
import torch.nn as nn
import torch.optim as optim

from datetime import datetime, timedelta
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
test_size = 7
seq_len = 7

In [ ]:
global input_size
input_size = 300
train_test_threshold = {'sample.json':(30,60),'classified_data_0.json':(100,425), \
	'classified_data_1.json':(100,440), 'classified_data_2.json':(60,150)}


In [ ]:
def feature_engineering(filename: str):
	global input_size
	with open(filename) as f:
		data = json.load(f)
	del data['上海市']
	
	total = 0
	useful = 0
	train_city = set()
	test_city = set()
	for city in data.values():
		total += len(city)
		if len(city) > train_test_threshold[filename][1]:
			test_city.add(city[0]['true_city'])
			train_city.add(city[0]['true_city'])
		elif len(city) > train_test_threshold[filename][0]:
			train_city.add(city[0]['true_city'])
		else:
			continue
		for p in city:
			if type(p['patient_vector']) == list:
				useful += 1	

	temp = np.empty((useful,input_size))
	temp_i = 0
	for city_string in train_city:
		city = data[city_string]
		for p in city:
			if type(p['patient_vector']) == list:
				temp[temp_i] = p['patient_vector']
				temp_i += 1

	pca = PCA(n_components=8)
	pca_result = pca.fit_transform(temp)

	temp_i = 0
	for city_string in train_city:
		city = data[city_string]
		for p in city:
			if type(p['patient_vector']) == list:
				p['patient_vector'] = pca_result[temp_i]
				temp_i += 1

	input_size = pca_result.shape[1]
	datas = {}
	for city_string in train_city:
		city = data[city_string]
		start_date = datetime.strptime(city[0]['time'],'%Y:%m:%d')
		end_date = datetime.strptime(city[-1]['time'],'%Y:%m:%d')
		period = (end_date-start_date).days+1
		CNT = np.zeros((period))
		VECTOR = np.zeros((period, input_size))
		cnt = 0
		vector = []
		day_idx = 0
		for p in city:
			if p['time'] != (start_date).strftime('%Y:%m:%d'):
				if any((type(v)==np.ndarray for v in vector)):
					vector = np.array([v for v in vector if type(v)==np.ndarray])
					VECTOR[day_idx] = np.mean(vector, axis=0)
				CNT[day_idx] = cnt
				cnt = 0
				vector = []
			while p['time'] != (start_date).strftime('%Y:%m:%d'):
				start_date += timedelta(days=1)
				day_idx += 1
			cnt += 1
			vector.append(p['patient_vector'])
		else:
			if any((type(v)==np.ndarray for v in vector)):
				vector = np.array([v for v in vector if type(v)==np.ndarray])
				VECTOR[day_idx] = np.mean(vector,axis=0)
			CNT[day_idx] = cnt
		temp = np.hstack((CNT.reshape(-1,1),VECTOR))
		datas[city_string] = temp	

	return datas, train_city, test_city

datas, train_city, test_city = feature_engineering('classified_data_0.json')

In [ ]:
def remove_toomany_0(rdata:np.ndarray, seq_len:int):
	move0 = []
	for i in range(len(rdata)-seq_len):
		if any(rdata[i:i+seq_len,0]):
			move0.append(1)
		else:
			move0.append(0)
	temp_i = 0
	size = seq_len+1
	for i in range(1,len(move0)):
		if move0[i-1] or move0[i]:
			size += 1
	new_data = np.zeros((size,len(rdata[0])))
	new_data[0] = rdata[0]
	for i in range(1,len(move0)):
		if move0[i-1] or move0[i]:
			temp_i += 1
			new_data[temp_i] = rdata[i]
	new_data[-seq_len:]	= rdata[-seq_len:]
	return new_data

for city in datas:
	temp = datas[city]
	temp = remove_toomany_0(datas[city],seq_len)
	datas[city] = torch.tensor(temp.reshape(-1,1,input_size+1)).float()	

In [ ]:
class Model_concat_gru(nn.Module):
    def __init__(self,input_size,hidden_size):
        super(Model_concat_gru, self).__init__()
        # concat
        self.gru = nn.GRU(input_size=input_size+1, \
            hidden_size=hidden_size)
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, x):
        output, _ = self.gru(x)
        output = output.squeeze()
        output = self.linear(output[-1]).squeeze()
        return output

In [ ]:
class Model_concat_deep_gru(nn.Module):
    def __init__(self,input_size,hidden_size):
        super(Model_concat_deep_gru, self).__init__()
        # concat
        self.gru = nn.GRU(input_size=input_size+1, \
            hidden_size=hidden_size,num_layers=2,dropout=0.2)
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, x):
        output, _ = self.gru(x)
        output = output.squeeze()
        output = self.linear(output[-1]).squeeze()
        return output
        

In [ ]:
class Model_add_gru(nn.Module):
    def __init__(self,input_size,hidden_size_1,hidden_size_2):
        super(Model_add_gru, self).__init__()
        # word embedding
        self.gru1 = nn.GRU(input_size=input_size, \
            hidden_size=hidden_size_1)
        # number
        self.gru2 = nn.GRU(input_size=1, \
            hidden_size=hidden_size_2)
        self.linear1= nn.Linear(hidden_size_1,1)
        self.linear2 = nn.Linear(hidden_size_2,1)

    def forward(self, x):
        x1, _ = self.gru1(x[:,:,1:])
        x2, _ = self.gru2(x[:,:,0:1])
        x1 = self.linear1(x1[-1]).squeeze()
        x2 = self.linear2(x2[-1]).squeeze()
        return x1+x2


In [ ]:
class Model_add_deep_gru(nn.Module):
    def __init__(self,input_size,hidden_size_1,hidden_size_2):
        super(Model_add_deep_gru, self).__init__()
        # word embedding
        self.gru1 = nn.GRU(input_size=input_size, \
            hidden_size=hidden_size_1,num_layers=2,dropout=0.2)
        # number
        self.gru2 = nn.GRU(input_size=1, \
            hidden_size=hidden_size_2,num_layers=2,dropout=0.2)
        self.linear1= nn.Linear(hidden_size_1,1)
        self.linear2 = nn.Linear(hidden_size_2,1)

    def forward(self, x):
        x1, _ = self.gru1(x[:,:,1:])
        x2, _ = self.gru2(x[:,:,0:1])
        x1 = self.linear1(x1[-1]).squeeze()
        x2 = self.linear2(x2[-1]).squeeze()
        return x1+x2
        

In [ ]:
def data_iter(data,seq_len,test_size,in_test):
	if in_test:
		indexs = np.arange(len(data)-seq_len-test_size)
	else:
		indexs = np.arange(len(data)-seq_len)
	random.shuffle(indexs)
	for i in indexs:
		yield data[i:i+seq_len],data[i+seq_len][0][0]
		

In [ ]:
def train(model, datas, test_size):
    # create your optimizer
    optimizer = optim.Adam(model.parameters(), lr=2e-3)
    criterion = nn.MSELoss()
    
    epoches = 100
    seq_len = 7
    for epoch in tqdm(range(epoches)):  # loop over the dataset multiple times

        running_loss = 0.0
        city_keys = list(datas.keys())
        random.shuffle(city_keys)
        for city in city_keys:
            
            for data in data_iter(datas[city],seq_len,test_size,city in test_city):
                # get the inputs; data is a list of [inputs, labels]
                inputs, labels = data
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward + backward + optimize
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

        if epoch%10 == 9:
            print(f'{epoch+1}: {running_loss:.2f}')

        running_loss = 0.0
    
    print('Finished Training')


In [ ]:
def test(model, datas, test_size):
	total_loss = 0.0
	for i in range(test_size, 0, -1):
		daily_loss = 0.0
		for city_string in test_city:
			inputs = datas[city_string][-i-seq_len:-i]
			outputs = model(inputs)
			y = float(outputs.relu())
			y_pred = [round(y)]
			y_true = [int(datas[city_string][-i][0][0])]
			print(city_string, f'{y:.3f}', y_pred[0], y_true[0])
			daily_loss += mean_squared_error(y_true, y_pred)
		print(daily_loss)
		total_loss += daily_loss
	print(total_loss)


In [ ]:
# based on 'sample.json'
# hidden_size = (4, 8, 12)
# test_size = 2
# for hz in hidden_size:
# 	model = Model_concat_gru(input_size, hz)
# 	train(model, datas, test_size)
# 	test(model, datas, test_size)
# for hz in hidden_size:
# 	model = Model_concat_deep_gru(input_size, hz)
# 	train(model, datas, test_size)
# 	test(model, datas, test_size)
# hidden_size = ((2,4), (2,8), (4,4), (4,8))
# for hz1,hz2 in hidden_size:
# 	model = Model_add_gru(input_size, hz1, hz2)
# 	train(model, datas, test_size)
# 	test(model, datas, test_size)
# for hz1,hz2 in hidden_size:
# 	model = Model_add_deep_gru(input_size, hz1, hz2)
# 	train(model, datas, test_size)
# 	test(model, datas, test_size)


In [ ]:
test_size = 7
model1 = Model_concat_gru(input_size, 8)
train(model1, datas, test_size)
test(model1, datas, test_size)


In [ ]:
model2 = Model_concat_deep_gru(input_size, 4)
train(model2, datas, test_size)
test(model2, datas, test_size)


In [ ]:
model3 = Model_add_gru(input_size, 2, 4)
train(model3, datas, test_size)
test(model3, datas, test_size)


In [ ]:
data_mse = [[5510.0, 4642.0, 3680.0], [5122.0, 4547.0, 4618.0], [461.0, 288.0, 240.0]]
data_rmse = [[np.sqrt(data_mse[i][j]/(5*7)) for i in range(3)] for j in range(3)]
data_rmse = np.array(data_rmse)


In [ ]:
sns.set_theme()
x = np.arange(3)
total_width, n = 0.8, 3
width = total_width/n
x = x - (total_width - width)/2
labels = ['data_0', 'data_1', 'data_2']
plt.bar(x, data_rmse[0], width, label='model_1')
plt.bar(x+width, data_rmse[1], width, label='model_2')
plt.bar(x+2*width, data_rmse[2], width, label='model_3')
plt.title('Preformance Evaluation By RMSE')
plt.ylabel('Root Mean Squared Error')
plt.xticks(np.arange(3), labels=labels)
plt.legend()
plt.savefig('result')
plt.show()
